Importamos todas las librerías que vamos a utilizar

In [203]:
import pandas as pd
import datetime as dt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import numpy as np
import matplotlib.pyplot as plt
import math

Creamos los datasets que vamos a utilizar, el de entrenamiento y el de test

In [190]:
dataset_train = pd.read_excel('bike_train.xlsx',index_col=0)
dataset_train.head(5)

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Verificamos el dataset 'dataset_train'

In [191]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11999 entries, 1 to 11999
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   dteday      11999 non-null  datetime64[ns]
 1   season      11999 non-null  int64         
 2   yr          11999 non-null  int64         
 3   mnth        11999 non-null  int64         
 4   hr          11999 non-null  int64         
 5   holiday     11999 non-null  int64         
 6   weekday     11999 non-null  int64         
 7   workingday  11999 non-null  int64         
 8   weathersit  11999 non-null  int64         
 9   temp        11999 non-null  float64       
 10  atemp       11999 non-null  float64       
 11  hum         11999 non-null  float64       
 12  windspeed   11999 non-null  float64       
 13  casual      11999 non-null  int64         
 14  registered  11999 non-null  int64         
 15  cnt         11999 non-null  int64         
dtypes: datetime64[ns](1), 

In [192]:
dataset_train.drop_duplicates()
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11999 entries, 1 to 11999
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   dteday      11999 non-null  datetime64[ns]
 1   season      11999 non-null  int64         
 2   yr          11999 non-null  int64         
 3   mnth        11999 non-null  int64         
 4   hr          11999 non-null  int64         
 5   holiday     11999 non-null  int64         
 6   weekday     11999 non-null  int64         
 7   workingday  11999 non-null  int64         
 8   weathersit  11999 non-null  int64         
 9   temp        11999 non-null  float64       
 10  atemp       11999 non-null  float64       
 11  hum         11999 non-null  float64       
 12  windspeed   11999 non-null  float64       
 13  casual      11999 non-null  int64         
 14  registered  11999 non-null  int64         
 15  cnt         11999 non-null  int64         
dtypes: datetime64[ns](1), 

In [193]:
dataset_train.describe()

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000,11999.000000
mean,2.210684,0.279523,5.530294,11.556630,0.027752,3.009334,0.684057,1.435036,0.465704,0.448888,0.625770,0.196433,30.049837,128.545129,158.594966
std,1.088798,0.448784,3.433005,6.911891,0.164269,2.005264,0.464910,0.650894,0.188576,0.169633,0.200637,0.125394,43.596915,124.692750,151.392952
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,1.000000,0.000000,3.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.320000,0.303000,0.470000,0.104500,3.000000,29.000000,34.000000
50%,2.000000,0.000000,5.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.460000,0.454500,0.620000,0.194000,13.000000,98.000000,118.000000
75%,3.000000,1.000000,8.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.620000,0.590900,0.790000,0.283600,39.000000,185.000000,232.000000
max,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,0.960000,1.000000,1.000000,0.850700,367.000000,770.000000,957.000000


Verificamos el dataset 'dataset_test'

Exploramos las correlaciones de las variables para mostrarnos los mejores indicadores para explicar la variable dependiente

In [197]:
## convertimos la columna 'dteday' (fecha) a ordinal para que pueda ser leída por corr() y para los modelos predictivos
dataset_train['dteday'] = dataset_train['dteday'].map(dt.datetime.toordinal)

In [198]:
dataset_train.corr()

,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,1.000000,0.119277,0.776468,0.168474,-0.009457,0.008254,0.007121,0.005410,-0.015217,0.030437,0.043644,-0.012197,-0.029317,0.115635,0.238038,0.229357
season,0.119277,1.000000,-0.446665,0.865268,-0.007610,-0.008928,-0.003688,0.012630,-0.009029,0.410790,0.413196,0.198937,-0.142924,0.119043,0.093595,0.111369
yr,0.776468,-0.446665,1.000000,-0.488103,-0.003948,0.001039,-0.002714,0.001068,-0.006309,-0.198929,-0.190360,-0.141319,0.067356,0.053371,0.171910,0.156960
mnth,0.168474,0.865268,-0.488103,1.000000,-0.007060,0.009372,0.014500,0.006441,-0.012239,0.353330,0.358541,0.201669,-0.147042,0.078563,0.061686,0.073431
hr,-0.009457,-0.007610,-0.003948,-0.007060,1.000000,0.000414,-0.004037,0.003454,-0.024003,0.139251,0.134633,-0.257202,0.130441,0.285182,0.381933,0.396698
holiday,0.008254,-0.008928,0.001039,0.009372,0.000414,1.000000,-0.102503,-0.248601,-0.014707,-0.028519,-0.038005,-0.036672,0.022681,0.022140,-0.044534,-0.030304
weekday,0.007121,-0.003688,-0.002714,0.014500,-0.004037,-0.102503,1.000000,0.031057,0.017259,-0.010002,-0.009841,-0.025932,0.008447,0.023687,0.019179,0.022618
workingday,0.005410,0.012630,0.001068,0.006441,0.003454,-0.248601,0.031057,1.000000,0.042202,0.062273,0.066305,0.022805,-0.002493,-0.298241,0.134789,0.025132
weathersit,-0.015217,-0.009029,-0.006309,-0.012239,-0.024003,-0.014707,0.017259,0.042202,1.000000,-0.080149,-0.081386,0.430378,0.009195,-0.147903,-0.130223,-0.149848
temp,0.030437,0.410790,-0.198929,0.353330,0.139251,-0.028519,-0.010002,0.062273,-0.080149,1.000000,0.991785,-0.019289,-0.013984,0.450777,0.322875,0.395743
